In [7]:
from dg_ml_core.collections import dict_ops
import numpy as np

In [29]:
def get_mean_std_string(key, dataset_stats):
    return str(np.round(dataset_stats['mean_' + key], 2)) + ' \\pm ' + str(np.round(dataset_stats['std_' + key], 2))

def get_grouped_number(number):
    if len(number) > 3:
        return f'{int(number):,}'.replace(",", "'")
#         grouped_number = ''
#         for digit, idx in enumerate(number[::-1]):
#             if int(idx) % 3 == 0 and int(idx) > 0:
#                 grouped_number += "'"
#             grouped_number += str(digit)
#         return grouped_number[::-1]
    else:
        return number

dataset_names = ['micro', 'midi', 'maxi']
stats_strings = dict()
for dataset in dataset_names:
    stats_path = f"gs://ma-muy/03_datasets/{dataset}_dataset/03_dataset_stats.json"
    dataset_stats = dict_ops.load_dict(stats_path)
    stats_strings[dataset] = []
    stats_strings[dataset].append(get_grouped_number(str(dataset_stats['num_users'])))
    stats_strings[dataset].append(get_grouped_number(str(dataset_stats['num_products'])))
    stats_strings[dataset].append(get_grouped_number(str(dataset_stats['num_events'])))
    stats_strings[dataset].append(get_grouped_number(str(dataset_stats['num_sessions'])))
    stats_strings[dataset].append(get_mean_std_string('sessions_per_user', dataset_stats))
    stats_strings[dataset].append(get_mean_std_string('events_per_product', dataset_stats))
    stats_strings[dataset].append(get_mean_std_string('events_per_session', dataset_stats))

headers = ['Property', 'MINI Dataset', 'MIDI Dataset', 'MAXI Dataset']
property_names = ['\\#Users', '\\#Products', '\\#Events', '\\#Sessions', '\\#Sessions per User', '\\#Events per Product', '\\#Events per Session']
print(' & '.join(headers) + ' \\\\ \\hline')
for a, b, c, d in zip(property_names, stats_strings['micro'], stats_strings['midi'], stats_strings['maxi']):
    print(f"{a} & {b} & {c} & {d} \\\\ \\hline")

/home/muy/.local/share/virtualenvs/ethz-msc-thesis-DnoiC4PD/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/muy/.local/share/virtualenvs/ethz-msc-thesis-DnoiC4PD/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For m

Property & MINI Dataset & MIDI Dataset & MAXI Dataset \\ \hline
\#Users & 23 & 15'242 & 242'797 \\ \hline
\#Products & 161 & 42'103 & 470'817 \\ \hline
\#Events & 633 & 1'234'697 & 28'726'701 \\ \hline
\#Sessions & 183 & 250'187 & 4'652'496 \\ \hline
\#Sessions per User & 7.96 \pm 5.41 & 16.41 \pm 22.08 & 19.16 \pm 28.67 \\ \hline
\#Events per Product & 3.93 \pm 5.84 & 29.33 \pm 69.98 & 61.27 \pm 263.3 \\ \hline
\#Events per Session & 3.46 \pm 0.82 & 4.94 \pm 3.07 & 6.17 \pm 60.22 \\ \hline


In [19]:
a = 'abcde'
print(a[::-1])

edcba
